In [1]:
import dlc_bci as bci

import torch
import numpy as np

from models import *
from callbacks import keep_best_model, store_best_model

from types import SimpleNamespace 


from torch import optim
from torch import nn
from torch import Tensor
from torch.autograd import Variable

%matplotlib inline
%load_ext autoreload
%autoreload 2

### Load

In [2]:
one_khz=False

train = SimpleNamespace()
train.X, train.y = bci.load(root='./data_bci', one_khz=one_khz)
print(str(type(train.X)), train.X.size())
print(str(type(train.y)), train.y.size())

test = SimpleNamespace()
test.X, test.y = bci.load(root='./data_bci', train=False, one_khz=one_khz)
print(str(type(test.X)), test.X.size())
print(str(type(test.y)), test.y.size())

<class 'torch.FloatTensor'> torch.Size([316, 28, 50])
<class 'torch.LongTensor'> torch.Size([316])
<class 'torch.FloatTensor'> torch.Size([100, 28, 50])
<class 'torch.LongTensor'> torch.Size([100])


In [3]:
        torch.manual_seed(0) 

In [6]:
from collections import OrderedDict

def cross_validate_grid_search(
    modelClass, 
    train, 
    epochs=100,
    n_splits=5,
    grid_search_on=OrderedDict([
        ("nb_hidden", [np.asscalar(n) for n in np.arange(40, 201, 40)]),
        ("activation", [nn.ReLU, nn.Tanh]),
        ("optimizer", [optim.Adam, optim.Adagrad, optim.Adamax]),
        ("weight_decay", [np.asscalar(wd) for wd in np.logspace(-6, -1, 4)])
    ])
):
    """ Apply a simplified grid search on the parameters in "grid_search_on" and optimize
    one at a time (to keep the complexity linear).    
    Finally save the best configuration.
    """
    
    X_tr, y_tr = modelClass.prepare_data(train)
    
    print("Optimizing in order:", grid_search_on.keys())
    # here we store the best parameters as we find them (initialized with just the first parameter)
    bestParams = {}
    for param in grid_search_on.keys():
        bestParams[param] = grid_search_on[param][0]
        
    
    for param_name in grid_search_on.keys():
        modelScores = {
            "tr_scores": [],
            "va_scores": []
        }
        param_values = grid_search_on[param_name]
        print("\nCross validation on", param_name+":")
        for n in param_values: # scan through the parameter values
            print("------", param_name, "=", str(n), "------")
            bestParams[param_name] = n
            model = modelClass(**bestParams)
            result = model.cross_validate(X_tr, y_tr, verbose=True, epochs=epochs, n_splits=n_splits)

            modelScores["tr_scores"].append(np.asscalar(np.mean(result["train_score"])))
            modelScores["va_scores"].append(np.asscalar(np.mean(result["test_score"])))
        return modelScores
        # save the obtained scores 
        model.save_data(modelScores, "cross_validation/"+param_name, pickle_protocol=0)
        # get the best parameter
        bestParams[param_name] = param_values[np.argmax(modelScores["va_scores"])]
        
    model.save_data(bestParams, "cross_validation/best_params", pickle_protocol=0)
    # [np.asscalar(wd) for wd in np.logspace(-6, -1, 4)]
    # best_n = n_hidden_values[np.argmax(modelScores["va_scores"])]
    # print('Best #hidden units:', best_n)
    # print('Test score:',
    #       CNN2D(n)
    #       .fit(X_tr, y_tr, epochs=50)
    #       .score(X_te, y_te))

In [7]:
asd = cross_validate_grid_search(CNN2D_MaxPool, train, epochs=2)

Optimizing in order: odict_keys(['nb_hidden', 'activation', 'optimizer', 'weight_decay'])

Cross validation on nb_hidden:
------ nb_hidden = 40 ------
----------------- fold 1/5 -----------------
Epoch 0: Train loss: 15.083938419818878. Train accuracy 55.95%. Test accuracy 45.31%
Epoch 1: Train loss: 9.115569114685059. Train accuracy 51.59%. Test accuracy 45.31%
----------------- fold 2/5 -----------------
Epoch 0: Train loss: 20.491197645664215. Train accuracy 46.64%. Test accuracy 58.73%
Epoch 1: Train loss: 9.313822567462921. Train accuracy 52.96%. Test accuracy 39.68%
----------------- fold 3/5 -----------------
Epoch 0: Train loss: 18.0937539935112. Train accuracy 59.68%. Test accuracy 53.97%
Epoch 1: Train loss: 9.469550848007202. Train accuracy 54.15%. Test accuracy 47.62%
----------------- fold 4/5 -----------------
Epoch 0: Train loss: 17.003881454467773. Train accuracy 49.01%. Test accuracy 55.56%
Epoch 1: Train loss: 9.507079780101776. Train accuracy 51.78%. Test accuracy 47

In [6]:
grid_search_on={
        "nb_hidden": [np.asscalar(n) for n in np.arange(40, 201, 40)],
        "activation": [nn.ReLU, nn.Tanh],
        "optimizer": [optim.Adam, optim.Adagrad, optim.Adamax],
        "weight_decay": [np.asscalar(wd) for wd in np.logspace(-6, -1, 4)],
    }
bestParams = {}
for param in grid_search_on.keys():
    bestParams[param] = grid_search_on[param][0]
bestParams

{'activation': torch.nn.modules.activation.ReLU,
 'nb_hidden': 40,
 'optimizer': torch.optim.adam.Adam,
 'weight_decays': 1e-06}